# Analise de Envelhecimento e Reforma
**Projeto 1: IBM HR Analytics - AiDAPT - Cegid Academy**

Envelhecimento da populacao e planeamento de sucessao.
Idade de reforma em Portugal: 67 anos.

Base de dados: Projeto1_IBM_HR

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from urllib.parse import quote_plus
from sqlalchemy import create_engine
load_dotenv(find_dotenv())

%load_ext sql

host = os.getenv('MSSQL_HOST', 'localhost')
port = os.getenv('MSSQL_PORT', '1433')
user = os.getenv('MSSQL_USER', 'sa')
password = quote_plus(os.getenv('MSSQL_PASSWORD', 'your_password_here'))
engine = create_engine(f"mssql+pymssql://{user}:{password}@{host}:{port}/Projeto1_IBM_HR")
%sql engine --alias Projeto1_IBM_HR

Tip: You may define configurations in /Users/diogosilva/Documents/Cegid_Academy/cegid_academy_aidapt/pyproject.toml or /Users/diogosilva/.jupysql/config.

Did not find user configurations in /Users/diogosilva/Documents/Cegid_Academy/cegid_academy_aidapt/pyproject.toml.

## 1. VISÃO GERAL DO ENVELHECIMENTO

### Estatísticas de idade

In [2]:
%%sql
SELECT
    MIN(Age) AS IdadeMinima,
    MAX(Age) AS IdadeMaxima,
    AVG(Age) AS IdadeMedia,
    STDEV(Age) AS DesvioPadrao
FROM Colaboradores;

Running query in 'Projeto1_IBM_HR'

IdadeMinima,IdadeMaxima,IdadeMedia,DesvioPadrao
18,60,36,9.135373489136729


### Distribuição por gerações

In [3]:
%%sql
SELECT
    CASE
        WHEN Age < 28 THEN 'Gen Z (< 28)'
        WHEN Age < 44 THEN 'Millennials (28-43)'
        WHEN Age < 60 THEN 'Gen X (44-59)'
        ELSE 'Baby Boomers (60+)'
    END AS Geracao,
    COUNT(*) AS Total,
    CAST(ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Colaboradores), 1) AS DECIMAL(5,1)) AS Percentagem,
    AVG(Age) AS IdadeMedia
FROM Colaboradores
GROUP BY
    CASE
        WHEN Age < 28 THEN 'Gen Z (< 28)'
        WHEN Age < 44 THEN 'Millennials (28-43)'
        WHEN Age < 60 THEN 'Gen X (44-59)'
        ELSE 'Baby Boomers (60+)'
    END
ORDER BY MIN(Age);

Running query in 'Projeto1_IBM_HR'

Geracao,Total,Percentagem,IdadeMedia
Gen Z (< 28),210,14.3,24
Millennials (28-43),913,62.1,34
Gen X (44-59),342,23.3,49
Baby Boomers (60+),5,0.3,60


## 2. COLABORADORES PERTO DA REFORMA

### Colaboradores com 55+ anos (próximos da reforma)

In [4]:
%%sql
SELECT
    EmployeeNumber,
    Age,
    (67 - Age) AS AnosParaReforma,
    Department,
    JobRole,
    JobLevel,
    YearsAtCompany,
    MonthlyIncome
FROM Colaboradores
WHERE Age >= 55
ORDER BY Age DESC;

Running query in 'Projeto1_IBM_HR'

EmployeeNumber,Age,AnosParaReforma,Department,JobRole,JobLevel,YearsAtCompany,MonthlyIncome
549,60,7,Research & Development,Manager,5,29,19566
573,60,7,Sales,Sales Executive,3,18,10266
732,60,7,Sales,Sales Executive,2,2,5405
1233,60,7,Sales,Sales Executive,2,11,5220
1697,60,7,Research & Development,Healthcare Representative,3,1,10883
1254,59,8,Sales,Sales Executive,2,6,5171
1283,59,8,Research & Development,Manufacturing Director,3,9,10512
1032,59,8,Research & Development,Manufacturing Director,4,5,13726
1048,59,8,Sales,Manager,3,6,11904
10,59,8,Research & Development,Laboratory Technician,1,1,2670


### Contagem por faixa etária crítica

In [5]:
%%sql
SELECT
    CASE
        WHEN Age >= 60 THEN '60+ (Reforma muito próxima: 0-7 anos)'
        WHEN Age >= 55 THEN '55-59 (Reforma próxima: 8-12 anos)'
        WHEN Age >= 50 THEN '50-54 (Pré-reforma: 13-17 anos)'
    END AS FaixaCritica,
    COUNT(*) AS Total,
    CAST(ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Colaboradores), 1) AS DECIMAL(5,1)) AS PercTotal
FROM Colaboradores
WHERE Age >= 50
GROUP BY
    CASE
        WHEN Age >= 60 THEN '60+ (Reforma muito próxima: 0-7 anos)'
        WHEN Age >= 55 THEN '55-59 (Reforma próxima: 8-12 anos)'
        WHEN Age >= 50 THEN '50-54 (Pré-reforma: 13-17 anos)'
    END
ORDER BY MIN(Age) DESC;

Running query in 'Projeto1_IBM_HR'

FaixaCritica,Total,PercTotal
60+ (Reforma muito próxima: 0-7 anos),5,0.3
55-59 (Reforma próxima: 8-12 anos),64,4.4
50-54 (Pré-reforma: 13-17 anos),104,7.1


## 3. ENVELHECIMENTO POR DEPARTAMENTO

### Idade média por departamento

In [6]:
%%sql
SELECT
    Department AS Departamento,
    COUNT(*) AS Total,
    AVG(Age) AS IdadeMedia,
    MIN(Age) AS IdadeMin,
    MAX(Age) AS IdadeMax,
    SUM(CASE WHEN Age >= 55 THEN 1 ELSE 0 END) AS Colaboradores55mais,
    CAST(ROUND(SUM(CASE WHEN Age >= 55 THEN 1.0 ELSE 0 END) * 100 / COUNT(*), 1) AS DECIMAL(5,1)) AS Perc55mais
FROM Colaboradores
GROUP BY Department
ORDER BY IdadeMedia DESC;

Running query in 'Projeto1_IBM_HR'

Departamento,Total,IdadeMedia,IdadeMin,IdadeMax,Colaboradores55mais,Perc55mais
Human Resources,63,37,19,59,4,6.3
Research & Development,961,37,18,60,47,4.9
Sales,446,36,18,60,18,4.0


### Distribuição etária por departamento

In [7]:
%%sql
SELECT
    Department AS Departamento,
    SUM(CASE WHEN Age < 30 THEN 1 ELSE 0 END) AS [18-29],
    SUM(CASE WHEN Age >= 30 AND Age < 40 THEN 1 ELSE 0 END) AS [30-39],
    SUM(CASE WHEN Age >= 40 AND Age < 50 THEN 1 ELSE 0 END) AS [40-49],
    SUM(CASE WHEN Age >= 50 AND Age < 60 THEN 1 ELSE 0 END) AS [50-59],
    SUM(CASE WHEN Age >= 60 THEN 1 ELSE 0 END) AS [60+],
    COUNT(*) AS Total
FROM Colaboradores
GROUP BY Department;

Running query in 'Projeto1_IBM_HR'

Departamento,18-29,30-39,40-49,50-59,60+,Total
Human Resources,13,25,17,8,0,63
Sales,96,200,102,45,3,446
Research & Development,217,397,230,115,2,961


## 4. ENVELHECIMENTO POR CARGO

### Idade média por cargo

In [8]:
%%sql
SELECT
    JobRole AS Cargo,
    COUNT(*) AS Total,
    AVG(Age) AS IdadeMedia,
    SUM(CASE WHEN Age >= 55 THEN 1 ELSE 0 END) AS Colaboradores55mais,
    CAST(ROUND(SUM(CASE WHEN Age >= 55 THEN 1.0 ELSE 0 END) * 100 / COUNT(*), 1) AS DECIMAL(5,1)) AS Perc55mais
FROM Colaboradores
GROUP BY JobRole
ORDER BY IdadeMedia DESC;

Running query in 'Projeto1_IBM_HR'

Cargo,Total,IdadeMedia,Colaboradores55mais,Perc55mais
Manager,102,46,17,16.7
Research Director,80,44,6,7.5
Healthcare Representative,131,39,11,8.4
Manufacturing Director,145,38,6,4.1
Sales Executive,326,36,12,3.7
Human Resources,52,35,1,1.9
Research Scientist,292,34,11,3.8
Laboratory Technician,259,34,5,1.9
Sales Representative,83,30,0,0.0


## 5. RISCO DE PERDA DE CONHECIMENTO

### Colaboradores seniores com muitos anos de experiência

In [9]:
%%sql
SELECT
    EmployeeNumber,
    Age,
    (67 - Age) AS AnosParaReforma,
    Department,
    JobRole,
    TotalWorkingYears AS ExperienciaTotal,
    YearsAtCompany AS AnosNaEmpresa,
    YearsInCurrentRole AS AnosNoCargo,
    JobLevel AS Nivel
FROM Colaboradores
WHERE Age >= 55
  AND (TotalWorkingYears >= 20 OR YearsAtCompany >= 15 OR JobLevel >= 4)
ORDER BY Age DESC, TotalWorkingYears DESC;

Running query in 'Projeto1_IBM_HR'

EmployeeNumber,Age,AnosParaReforma,Department,JobRole,ExperienciaTotal,AnosNaEmpresa,AnosNoCargo,Nivel
549,60,7,Research & Development,Manager,33,29,8,5
573,60,7,Sales,Sales Executive,22,18,13,3
140,59,8,Human Resources,Manager,30,3,2,5
1032,59,8,Research & Development,Manufacturing Director,30,5,3,4
81,59,8,Sales,Sales Executive,28,21,16,3
1283,59,8,Research & Development,Manufacturing Director,25,9,7,3
91,59,8,Sales,Sales Executive,20,4,3,2
165,58,9,Research & Development,Healthcare Representative,40,40,10,3
825,58,9,Research & Development,Research Director,40,31,15,5
131,58,9,Sales,Sales Executive,38,37,10,4


### Análise de risco por departamento

In [10]:
%%sql
SELECT
    Department AS Departamento,
    COUNT(*) AS ColaboradoresEmRisco,
    AVG(TotalWorkingYears) AS MediaExperiencia,
    AVG(YearsAtCompany) AS MediaAnosEmpresa,
    SUM(CASE WHEN JobLevel >= 4 THEN 1 ELSE 0 END) AS SenioresAltoNivel
FROM Colaboradores
WHERE Age >= 55
GROUP BY Department
ORDER BY ColaboradoresEmRisco DESC;

Running query in 'Projeto1_IBM_HR'

Departamento,ColaboradoresEmRisco,MediaExperiencia,MediaAnosEmpresa,SenioresAltoNivel
Research & Development,47,20,9,18
Sales,18,23,11,8
Human Resources,4,27,5,3


## 6. PLANEAMENTO DE SUCESSÃO

### Cargos com colaboradores perto da reforma e sem substitutos óbvios

In [11]:
%%sql
SELECT
    JobRole AS Cargo,
    SUM(CASE WHEN Age >= 55 THEN 1 ELSE 0 END) AS PertoDaReforma,
    SUM(CASE WHEN Age < 35 THEN 1 ELSE 0 END) AS Jovens,
    COUNT(*) AS Total,
    CASE
        WHEN SUM(CASE WHEN Age >= 55 THEN 1 ELSE 0 END) > SUM(CASE WHEN Age < 35 THEN 1 ELSE 0 END)
        THEN 'RISCO: Mais seniores que jovens'
        ELSE 'OK'
    END AS AlertaSucessao
FROM Colaboradores
GROUP BY JobRole
HAVING SUM(CASE WHEN Age >= 55 THEN 1 ELSE 0 END) > 0
ORDER BY PertoDaReforma DESC;

Running query in 'Projeto1_IBM_HR'

Cargo,PertoDaReforma,Jovens,Total,AlertaSucessao
Manager,17,7,102,RISCO: Mais seniores que jovens
Sales Executive,12,148,326,OK
Research Scientist,11,174,292,OK
Healthcare Representative,11,36,131,OK
Research Director,6,13,80,OK
Manufacturing Director,6,48,145,OK
Laboratory Technician,5,139,259,OK
Human Resources,1,25,52,OK


### Pirâmide etária por nível hierárquico

In [12]:
%%sql
SELECT
    JobLevel AS Nivel,
    CASE JobLevel
        WHEN 1 THEN 'Entry Level'
        WHEN 2 THEN 'Junior'
        WHEN 3 THEN 'Mid-Level'
        WHEN 4 THEN 'Senior'
        WHEN 5 THEN 'Executive'
    END AS DescricaoNivel,
    SUM(CASE WHEN Age < 35 THEN 1 ELSE 0 END) AS [<35],
    SUM(CASE WHEN Age >= 35 AND Age < 50 THEN 1 ELSE 0 END) AS [35-49],
    SUM(CASE WHEN Age >= 50 THEN 1 ELSE 0 END) AS [50+],
    AVG(Age) AS IdadeMedia
FROM Colaboradores
GROUP BY JobLevel
ORDER BY JobLevel;

Running query in 'Projeto1_IBM_HR'

Nivel,DescricaoNivel,<35,35-49,50+,IdadeMedia
1,Entry Level,353,162,28,32
2,Junior,237,262,35,36
3,Mid-Level,56,128,34,40
4,Senior,5,54,47,47
5,Executive,0,40,29,47


## 7. EXPERIÊNCIA E CONHECIMENTO INSTITUCIONAL

### Anos médios na empresa por faixa etária

In [13]:
%%sql
SELECT
    CASE
        WHEN Age < 30 THEN '18-29'
        WHEN Age < 40 THEN '30-39'
        WHEN Age < 50 THEN '40-49'
        WHEN Age < 60 THEN '50-59'
        ELSE '60+'
    END AS FaixaEtaria,
    COUNT(*) AS Total,
    AVG(YearsAtCompany) AS MediaAnosEmpresa,
    AVG(YearsInCurrentRole) AS MediaAnosCargo,
    AVG(TotalWorkingYears) AS MediaExperienciaTotal
FROM Colaboradores
GROUP BY
    CASE
        WHEN Age < 30 THEN '18-29'
        WHEN Age < 40 THEN '30-39'
        WHEN Age < 50 THEN '40-49'
        WHEN Age < 60 THEN '50-59'
        ELSE '60+'
    END
ORDER BY MIN(Age);

Running query in 'Projeto1_IBM_HR'

FaixaEtaria,Total,MediaAnosEmpresa,MediaAnosCargo,MediaExperienciaTotal
18-29,326,4,2,4
30-39,622,6,4,9
40-49,349,8,4,15
50-59,168,9,4,21
60+,5,12,6,19


## 8. ATTRITION POR FAIXA ETÁRIA

### Quem está a sair por idade?

In [14]:
%%sql
SELECT
    CASE
        WHEN Age < 30 THEN '18-29'
        WHEN Age < 40 THEN '30-39'
        WHEN Age < 50 THEN '40-49'
        ELSE '50+'
    END AS FaixaEtaria,
    Attrition,
    COUNT(*) AS Total,
    CAST(ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY
        CASE
            WHEN Age < 30 THEN '18-29'
            WHEN Age < 40 THEN '30-39'
            WHEN Age < 50 THEN '40-49'
            ELSE '50+'
        END
    ), 1) AS DECIMAL(5,1)) AS PercNaFaixa
FROM Colaboradores
GROUP BY
    CASE
        WHEN Age < 30 THEN '18-29'
        WHEN Age < 40 THEN '30-39'
        WHEN Age < 50 THEN '40-49'
        ELSE '50+'
    END,
    Attrition
ORDER BY MIN(Age), Attrition;

Running query in 'Projeto1_IBM_HR'

FaixaEtaria,Attrition,Total,PercNaFaixa
18-29,No,235,72.1
18-29,Yes,91,27.9
30-39,No,533,85.7
30-39,Yes,89,14.3
40-49,No,315,90.3
40-49,Yes,34,9.7
50+,No,150,86.7
50+,Yes,23,13.3


## 9. RESUMO EXECUTIVO - ENVELHECIMENTO

In [15]:
%%sql
SELECT '=== RESUMO ENVELHECIMENTO ===' AS Info;

Running query in 'Projeto1_IBM_HR'

Info
=== RESUMO ENVELHECIMENTO ===


### Números chave

In [16]:
%%sql
SELECT
    'Total 55+ anos' AS Metrica,
    COUNT(*) AS Valor,
    CAST(CAST(ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Colaboradores), 1) AS DECIMAL(5,1)) AS VARCHAR(10)) + '%' AS Percentagem
FROM Colaboradores
WHERE Age >= 55
UNION ALL
SELECT
    'Total 60+ anos',
    COUNT(*),
    CAST(CAST(ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Colaboradores), 1) AS DECIMAL(5,1)) AS VARCHAR(10)) + '%'
FROM Colaboradores
WHERE Age >= 60
UNION ALL
SELECT
    'Reforma nos próximos 5 anos (62+)',
    COUNT(*),
    CAST(CAST(ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Colaboradores), 1) AS DECIMAL(5,1)) AS VARCHAR(10)) + '%'
FROM Colaboradores
WHERE Age >= 62;

Running query in 'Projeto1_IBM_HR'

Metrica,Valor,Percentagem
Total 55+ anos,69,4.7%
Total 60+ anos,5,0.3%
Reforma nos próximos 5 anos (62+),0,0.0%


### Departamento mais envelhecido

In [17]:
%%sql
SELECT TOP 1
    'Departamento mais envelhecido' AS Metrica,
    Department AS Departamento,
    CAST(AVG(Age) AS VARCHAR(10)) + ' anos média' AS IdadeMedia
FROM Colaboradores
GROUP BY Department
ORDER BY AVG(Age) DESC;

Running query in 'Projeto1_IBM_HR'

Metrica,Departamento,IdadeMedia
Departamento mais envelhecido,Human Resources,37 anos média


### Alerta de conhecimento em risco

In [18]:
%%sql
SELECT
    'Colaboradores seniores (55+) com 20+ anos experiência' AS Alerta,
    COUNT(*) AS Total
FROM Colaboradores
WHERE Age >= 55 AND TotalWorkingYears >= 20;

Running query in 'Projeto1_IBM_HR'

Alerta,Total
Colaboradores seniores (55+) com 20+ anos experiência,39
